In [14]:
import pandas as pd
import os
import json
import warnings
warnings.filterwarnings("ignore")

In [15]:
guan_id = [500106003, 500101022, 500101181, 500106004]

In [16]:
root_dir = 'C:\\Users\\GD.Lien\\Desktop\\UBIKE_data\\html.2023.final.data-release\\release'

In [17]:
# 確保 DataFrame 中每天的每一分鐘都有資料。
def ensure_data_for_every_minute(df):
    unique_dates = df['timestamp'].unique()
    full_df = pd.DataFrame()

    for date in unique_dates:
        day_df = df[df['timestamp'] == date]
        all_times = pd.date_range('00:00', '23:59', freq='T').strftime('%H:%M')
        time_df = pd.DataFrame(all_times, columns=['time'])
        time_df['timestamp'] = date

        merged_df = pd.merge(time_df, day_df, on=['time', 'timestamp'], how='left')
        full_df = pd.concat([full_df, merged_df], ignore_index=True)

    return full_df

In [18]:
# 遍歷給定目錄及其子目錄，尋找並讀取指定的 JSON 文件，然後將其轉換為 pandas DataFrame。
def collect_json_data(root_dir, staion_id):
    target_file = f'{staion_id}.json'
    all_data = pd.DataFrame()

    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file == target_file:
                file_path = os.path.join(subdir, file)

                with open(file_path, 'r') as json_file:
                    data = json.load(json_file)

                    df = pd.DataFrame.from_dict(data, orient='index')
                    df['timestamp'] = os.path.basename(subdir)
                    df.reset_index(inplace=True)
                    df.rename(columns={'index': 'time'}, inplace=True)
                    # 確保 DataFrame 中每天的每一分鐘都有資料。
                    df = ensure_data_for_every_minute(df)
                    df[['tot', 'sbi', 'bemp', 'act']] = df[['tot', 'sbi', 'bemp', 'act']].fillna(method='ffill')
                    df[['tot', 'sbi', 'bemp', 'act']] = df[['tot', 'sbi', 'bemp', 'act']].fillna(method='bfill')
                    all_data = pd.concat([all_data, df], ignore_index=True)
    return all_data

In [21]:
for id in guan_id:
    staion_id = id 
    df = collect_json_data(root_dir, staion_id)
    df['staion_id'] = staion_id
    output_path = f'{staion_id}_data.csv'
    df.to_csv(output_path, index=False)
    print(df.shape)
    print(f"Data saved to {output_path}")

(79200, 7)
Data saved to 500106003_data.csv
(79200, 7)
Data saved to 500101022_data.csv
(79200, 7)
Data saved to 500101181_data.csv
(79200, 7)
Data saved to 500106004_data.csv


In [20]:
df

,time,timestamp,tot,sbi,bemp,act,staion_id
0,00:00,20231002,12.0,1.0,11.0,1,500106004
1,00:01,20231002,12.0,1.0,11.0,1,500106004
2,00:02,20231002,12.0,1.0,11.0,1,500106004
3,00:03,20231002,12.0,1.0,11.0,1,500106004
4,00:04,20231002,12.0,1.0,11.0,1,500106004
...,...,...,...,...,...,...,...
79195,23:55,20231202,12.0,3.0,9.0,1,500106004
79196,23:56,20231202,12.0,3.0,9.0,1,500106004
79197,23:57,20231202,12.0,3.0,9.0,1,500106004
79198,23:58,20231202,12.0,3.0,9.0,1,500106004
